Importamos la librerias

In [246]:
from mesa import Agent, Model
from mesa.time import SimultaneousActivation
from mesa.space import SingleGrid
from mesa.datacollection import DataCollector
from mesa.batchrunner import BatchRunner

# matplotlib lo vamos a utilizar para realizar una ''animación'' de mi modelo.
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd

# Definimos otros paquete que vamos a usar para medir el tiempo de ejecución del modelo
import time
import datetime

In [247]:
def get_grid(model):
    grid = np.zeros((model.grid.height, model.grid.width))
    for ren in range(model.grid.height):
        for col in range(model.grid.width):
            if model.grid.is_cell_empty((col, ren)):
                grid[ren][col] = 0
            else:
                grid[ren][col] = 1
            
    return grid

Definimos el agente 

In [248]:
class CarAgent(Agent):
    def __init__(self, unique_id, model,x,y,isBroken):
        super().__init__(unique_id, model)
        #Position
        self.x=int(x)
        self.y=int(y)
        #Velocity
        self.xvelocity=0
        self.yvelocity=5
        self.visionRange=3
        self.isBroken=isBroken
        
        
    def move(self):
        self.y=int(self.yvelocity+self.y)
        self.x=int(self.x+self.xvelocity)
        self.model.grid.move_agent(self, (self.x,self.y))
        
    def canCrash(self):
        limit=self.y+self.visionRange*self.yvelocity
        for agent in model.schedule.agents:
            if(agent.x==self.x and agent.y>self.y and agent.y<=limit):
                return True
        return False
    
    def brake(self):
        self.xvelocity=int(self.xvelocity/2)
        self.yvelocity=int(self.yvelocity/2)
        
    def step(self):
        if (self.canCrash()):
            print("There is a car in the next")
            print(self.visionRange*self.yvelocity)
            self.brake()
        print("POS")
        print(self.x)
        print(self.y)
        print("VEL")
        print(self.xvelocity)
        print(self.yvelocity)
        self.move()

 

Definimos el modelo

In [249]:
class highwayModel(Model):
    def __init__(self,n, widht, height):
        self.num_agents = n
        self.grid=SingleGrid(widht,height,True)
        self.schedule = SimultaneousActivation(self)
        
        for i in range(self.num_agents):
        
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
        
            a = CarAgent(i, self,x,y,0)
            self.schedule.add(a)  
            self.grid.place_agent(a, (x, y))
        
        self.createBarrier()
        self.datacollector = DataCollector(model_reporters={"Grid": get_grid})
        

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()
    
    def createBarrier(self):
         for x in range(model.grid.width):
                a = CarAgent(i, self,x,self.grid.width-2,1)
                self.grid.place_agent(a, (x,self.grid.width-2))
                
        
    

In [250]:
GRID_SIZE= 100
MAX_GENERATION= 40
NUM_CARS=3
model=highwayModel(NUM_CARS,GRID_SIZE,GRID_SIZE)
for i in range (MAX_GENERATION):
    model.step()
    


POS
7
33
VEL
0
5
POS
13
40
VEL
0
5
POS
4
92
VEL
0
5
POS
7
38
VEL
0
5
POS
13
45
VEL
0
5
POS
4
97
VEL
0
5
POS
7
43
VEL
0
5
POS
13
50
VEL
0
5
POS
4
102
VEL
0
5
POS
7
48
VEL
0
5
POS
13
55
VEL
0
5
POS
4
107
VEL
0
5
POS
7
53
VEL
0
5
POS
13
60
VEL
0
5
POS
4
112
VEL
0
5
POS
7
58
VEL
0
5
POS
13
65
VEL
0
5
POS
4
117
VEL
0
5
POS
7
63
VEL
0
5
POS
13
70
VEL
0
5
POS
4
122
VEL
0
5
POS
7
68
VEL
0
5
POS
13
75
VEL
0
5
POS
4
127
VEL
0
5
POS
7
73
VEL
0
5
POS
13
80
VEL
0
5
POS
4
132
VEL
0
5
POS
7
78
VEL
0
5
POS
13
85
VEL
0
5
POS
4
137
VEL
0
5
POS
7
83
VEL
0
5
POS
13
90
VEL
0
5
POS
4
142
VEL
0
5
POS
7
88
VEL
0
5
POS
13
95
VEL
0
5
POS
4
147
VEL
0
5
POS
7
93
VEL
0
5


Exception: Cell not empty

In [ ]:
all_grid = model.datacollector.get_model_vars_dataframe()

Graficar la informacion

In [ ]:
# Graficamos la información usando `matplotlib`

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.cm.binary)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    #print(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=MAX_GENERATION)

In [251]:
anim